# Messages Summarization
![Chain](images/summarization.png)

## "Standard" chatbot

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display

# OPENAI_API_KEY environment variable must be set
llm = ChatOpenAI(model="gpt-4o-mini")

# Defining Schema
##################################################################################
class State(MessagesState):
    question: str
    answer: str


# Defining Agent's node
##################################################################################

# System message
chatbot_system_message = SystemMessage(content=("""
You are a helpful and knowledgeable chatbot assistant. 
Your goal is to provide clear and accurate answers to user questions based on the information they provide. 
Stay focused, concise, and ensure your responses are relevant to the context of the conversation. 
If you don’t have enough information, ask for clarification.”
"""))

# Node
def chatbot(state: State) -> State:
    question = HumanMessage(content=state.get("question", ""))
    response = llm.invoke([chatbot_system_message] + state["messages"] + [question]);

    return State(
        messages = [question, response],
        question = state.get("question", None),
        answer = response.content
    )


# Defining Graph
##################################################################################

builder = StateGraph(State)
builder.add_node("chatbot", chatbot)


# Define edges: these determine how the control flow moves
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

memory = MemorySaver()
chatbot_graph = builder.compile(checkpointer=memory)

# Show
display(Image(chatbot_graph.get_graph(xray=True).draw_mermaid_png()))

## Removing messages

1) we use MessagesState
2) MessageState has a built-in list of messages ("messages" key)
3) Key "messages" has a built-in reducer "add_messages"

In [ ]:
from typing import Annotated
from langgraph.graph.message import add_messages

class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.graph.message import add_messages

messages = [
    HumanMessage(content="Message 1", id="1"),
    AIMessage(content="Message 2", id="2"),
    HumanMessage(content="Message 3", id="3"),
    AIMessage(content="Message 4", id="4"),
    HumanMessage(content="Message 5", id="5"),
    AIMessage(content="Message 6", id="6")
]

new_message = HumanMessage(content="Message 7", id="7")

# Test
messages = add_messages(messages , new_message)
messages

In [ ]:
from langchain_core.messages import RemoveMessage

# Isolate messages to delete
delete_messages = [RemoveMessage(id=m.id) for m in messages[:-2]]
delete_messages

In [ ]:
messages = add_messages(messages , delete_messages)
messages

## Summarization chatbot

In [ ]:
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END

# Defining Schema
##################################################################################
class SummaryState(State):
    summary: str

# Nodes
def chatbot(state: SummaryState) -> SummaryState:
    summary = state.get("summary", "") # getting summary if it exists

    # If there is summary, then we add it
    if summary:
        # define summary as SystemMessage
        summary_message = SystemMessage(content=(f"""
        Summary of Conversation:

        {summary}
        """))

        messages_with_summary = [summary_message] + state["messages"]
    
    else:
        messages_with_summary = state["messages"]


    question = HumanMessage(content=state.get("question", ""))

    response = llm.invoke([chatbot_system_message] + messages_with_summary + [question]);

    return SummaryState(
        messages = [question, response],
        question = state.get("question", None),
        answer = response.content,
        summary = state.get("summary", None)
    )


def summarize(state: SummaryState) -> SummaryState:
    summary = state.get("summary", "")
    # no system message
    # the order of components is important

    if summary:
        summary_message = HumanMessage(content=(f"""
            Expand the summary below by incorporating the above conversation while preserving context, key points, and 
            user intent. Rework the summary if needed. Ensure that no critical information is lost and that the 
            conversation can continue naturally without gaps. Keep the summary concise yet informative, removing 
            unnecessary repetition while maintaining clarity.
            
            Only return the updated summary. Do not add explanations, section headers, or extra commentary.

            Existing summary:

            {summary}
            """)
        )
        
    else:
        summary_message = HumanMessage(content="""
        Summarize the above conversation while preserving full context, key points, and user intent. Your response 
        should be concise yet detailed enough to ensure seamless continuation of the discussion. Avoid redundancy, 
        maintain clarity, and retain all necessary details for future exchanges.

        Only return the summarized content. Do not add explanations, section headers, or extra commentary.
        """)

    # Add prompt to our history
    messages = state["messages"] + [summary_message]
    response = llm.invoke(messages)
    
    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    
    return SummaryState(
        messages = delete_messages,
        question = state.get("question", None),
        answer = state.get("answer", None),
        summary = response.content
    )


# Edges

# Determine whether to end or summarize the conversation
def should_summarize(state: SummaryState):
    messages = state["messages"]
    
    if len(messages) > 2:
        return "summarize"
    
    return END


# Graph
workflow = StateGraph(SummaryState)
workflow.add_node(chatbot)
workflow.add_node(summarize)

workflow.add_edge(START, "chatbot")
workflow.add_conditional_edges("chatbot", should_summarize)
workflow.add_edge("summarize", END)


memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
thread_id = "1"
config = {"configurable": {"thread_id": thread_id}}

graph.invoke(State(question="Hi, I’m working on a Python project, and I’m stuck with handling API responses."), config)

In [ ]:
graph.invoke(State(question="Sorry what was my previous question?"), config)

In [ ]:
graph.invoke(State(question="Ahh, yeah right! So I’m mostly struggling with parsing JSON responses. Sometimes the structure isn’t what I expect, and it breaks my code."), config)

In [ ]:
graph.invoke(State(question="Got it! That helps a lot. What would be the best way to handle deeply nested JSON data when I only need a few specific values?"), config)

In [ ]:
graph.invoke(State(question="How is the weather outside?"), config)